In [5]:
%%time

## Import packages
import swat
import os
import pandas as pd


## Please specify your specific connection to CAS
conn = swat.CAS()

## Connect to SAS Viya (This works in Viya for learners)
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue,port=portValue,password=passwordToken)


## Set options
pd.set_option('display.max_columns', None)


## Load action sets
conn.loadActionSet('freqTab')
conn.loadActionSet('fedSQL')

## Add a caslib
conn.addCaslib(name='ate_cas', path=r'/shared/home/Peter.Styliadis@sas.com/ate_path')


## View available files in a caslib
display(conn.fileInfo(caslib='ate_cas'))


## Load data into CAS memory
loadFilesIntoMemory = ['orders_demo.sashdat', 'discount_lookup.sas7bdat']   ## <---Use the sashdat file
for file in loadFilesIntoMemory:
    conn.loadTable(path = file, 
                   caslib = 'ate_cas', 
                   casout = {'caslib':'ate_cas', 
                             'replace':True})

    
## View available CAS tables
display(conn.tableInfo(caslib='ate_cas'))

    
## Reference the orders_demo CAS table
ordersTbl = conn.CASTable('orders_demo', caslib='ate_cas')


## Explocre the orders_demo table
display(ordersTbl.head())
display(ordersTbl.summary())


## Execute the data step code
ds_code = """
data ate_cas.orders_demo_calc_columns;
    set ate_cas.orders_demo;
    Year = year(OrderDate);
    Month = Month(OrderDate);
    TotalCost = Quantity * Cost;
    TotalPrice = Quantity * Price;    
    Profit = TotalPrice - TotalCost;
    pctProfit = Profit / TotalCost;
    if Return='' then Return='No';
    format pctProfit percent7.2
           Price Cost TotalPrice TotalCost Profit dollar28.2;
    run;
"""
display(conn.runCode(code=ds_code))


## Reference the new table created from the data step
orders_demo_calc_columns = conn.CASTable('orders_demo_calc_columns', caslib='ate_cas')


## View frequency tables of the new CAS table 
display(orders_demo_calc_columns.freqTab(tabulate = ['Product', 'Country', 'DiscountCode', 'Return', 'Year']))


## Preview the new CAS table
display(orders_demo_calc_columns.head(10))


## Reference and preview the discount lookup table
discount_table = conn.CASTable('discount_lookup', caslib='ate_cas')
display(discount_table.head(10))


## Join the data using FedSQL
join_query = """
    create table ate_cas.orders_demo_final as
    select f.*, 
           l.pct_discount * .01 as pctDiscount, 
           l.discount_description
        from ate_cas.orders_demo_calc_columns as f left join 
             ate_cas.discount_lookup as l
        on f.DiscountCode = l.discountCode;
"""
display(conn.execDirect(query = join_query))


## Preview the final data
orders_final = conn.CASTable('orders_demo_final', caslib='ate_cas')
display(orders_final.head())


## Summarize totals by Country and Year
display(conn.summary(table={'name':'orders_demo_final',
                            'caslib':'ate_cas',
                            'groupby':{'Country','Year'}},
                       subset='SUM',
                       inputs={'TotalCost','TotalPrice','Profit'},
                       casout={'name':'orders_summary', 'caslib':'ate_cas'})
)

display(conn.fetch(table={'name':'orders_summary', 'caslib':'ate_cas'}))

conn.terminate()

NOTE: Added action set 'freqTab'.
NOTE: Added action set 'fedSQL'.
NOTE: 'ate_cas' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'ate_cas'.


,Permission,Owner,Group,Name,Size,Encryption,Time,ModTime
0,-rwxr-xr-x,Peter.Styliadis@sas.com,v4e_users,orders_demo.sashdat,1440534880,NONE,2022-06-24T09:10:56-04:00,1.971695e+09
1,-rw-r--r--,Peter.Styliadis@sas.com,v4e_users,orders_demo.sas7bdat,1443627008,,2022-06-24T09:10:41-04:00,1.971695e+09
2,-rw-r--r--,Peter.Styliadis@sas.com,v4e_users,discount_lookup.sas7bdat,131072,,2022-06-24T09:10:41-04:00,1.971695e+09


NOTE: Cloud Analytic Services made the file orders_demo.sashdat available as table ORDERS_DEMO in caslib ate_cas.
NOTE: Cloud Analytic Services made the file discount_lookup.sas7bdat available as table DISCOUNT_LOOKUP in caslib ate_cas.


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,ORDERS_DEMO,10000000,9,0,utf-8,2022-06-24T09:37:45-04:00,2022-06-24T09:37:45-04:00,2022-06-24T09:37:45-04:00,UTF8,1.971697e+09,1.971697e+09,1.971697e+09,0,0,0,0,orders_demo.sashdat,ate_cas,0,Peter.Styliadis@sas.com,,2022-06-24T09:10:56-04:00,1.971695e+09
1,DISCOUNT_LOOKUP,7,3,0,utf-8,2022-06-24T09:37:45-04:00,2022-06-24T09:37:45-04:00,2022-06-24T09:37:45-04:00,UTF8,1.971697e+09,1.971697e+09,1.971697e+09,0,0,0,0,discount_lookup.sas7bdat,ate_cas,0,Peter.Styliadis@sas.com,,2022-06-24T09:10:41-04:00,1.971695e+09


,Product,Country,OrderDate,DiscountCode,Return,StoreID,Quantity,Price,Cost
0,Pants,EN,2020-06-27,FREEDEAL,,19.0,330.0,7.64,1.49
1,Shirts,US,2020-09-19,,,31.0,440.0,6.39,1.99
2,Shirts,CA,2019-12-01,,,20.0,265.0,7.19,1.99
3,Sweatshirt,EN,2021-10-10,,,12.0,210.0,9.89,1.99
4,Pants,CA,2021-11-17,TC10,,27.0,260.0,8.09,1.49


...

+ Elapsed: 0.978s, user: 14.8s, sys: 1.69s, mem: 234mb

,Product,Country,OrderDate,DiscountCode,Return,StoreID,Quantity,Price,Cost,Year,Month,TotalCost,TotalPrice,Profit,pctProfit,PCTDISCOUNT,Discount_Description
0,Shirts,US,2018-04-06,,No,21.0,100.0,7.99,1.99,2018.0,4.0,199.00,799.00,600.00,3.015075,NaN,
1,Hats,GR,2021-01-31,,No,15.0,395.0,4.24,0.99,2021.0,1.0,391.05,1674.80,1283.75,3.282828,NaN,
2,Sweatshirt,AU,2020-03-18,,No,21.0,165.0,10.44,1.99,2020.0,3.0,328.35,1722.60,1394.25,4.246231,NaN,
3,Sweatshirt,US,2021-08-07,,No,14.0,75.0,10.99,1.99,2021.0,8.0,149.25,824.25,675.00,4.522613,NaN,
4,Hats,AU,2021-10-13,BB20,No,25.0,130.0,4.74,0.99,2021.0,10.0,128.70,616.20,487.50,3.787879,0.2,TV special discount


,casLib,Name,Rows,Columns,casTable
0,ate_cas,orders_summary,75,6,"CASTable('orders_summary', caslib='ate_cas')"


,Year,Year_f,Country,Country_f,_Column_,_Sum_
0,2017.0,2017,AU,AU,Profit,1.469643e+08
1,2017.0,2017,AU,AU,TotalCost,4.529027e+07
2,2017.0,2017,AU,AU,TotalPrice,1.922546e+08
3,2017.0,2017,CA,CA,Profit,2.938536e+08
4,2017.0,2017,CA,CA,TotalCost,9.045042e+07
5,2017.0,2017,CA,CA,TotalPrice,3.843040e+08
6,2017.0,2017,EN,EN,Profit,1.472425e+08
7,2017.0,2017,EN,EN,TotalCost,4.533184e+07
8,2017.0,2017,EN,EN,TotalPrice,1.925744e+08
9,2017.0,2017,GR,GR,Profit,2.932690e+08


CPU times: user 563 ms, sys: 65 ms, total: 628 ms
Wall time: 3.99 s
